# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.1016/j.dsx.2020.04.012           2
10.31729/jnma.5498                  2
10.1016/j.virol.2005.12.006         1
10.1128/mra.00383-20                1
10.1016/j.annonc.2020.08.1812       1
                                   ..
10.1016/j.earlhumdev.2020.105258    1
10.1111/irv.12433                   1
10.1101/2020.10.23.20218214         1
10.1007/s00134-020-06226-9          1
10.1038/s41583-021-00428-w          1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
18471,"[{'affiliation-city': 'Rehovot', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85100044416', ..."
18472,NaN,"{'srctype': 'j', 'eid': '2-s2.0-33645723465', ..."
18473,"[{'affiliation-city': 'Rockville', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85096082670', ..."
18474,None,None


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

18476

In [ ]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 18476 -> 10.1101/2020.03.30.015164
Position: 18477 -> 10.1155/2020/2396569
Position: 18478 -> 10.1177/0333102420965139
Position: 18479 -> 10.3389/fneur.2020.00806
Position: 18480 -> 10.1007/978-1-59745-181-9_20
Position: 18481 -> 10.1007/s11356-020-10932-8
Position: 18482 -> 10.1371/journal.pntd.0001359
Position: 18483 -> 10.1038/s41467-020-20608-z
Position: 18484 -> 10.1007/s40692-020-00181-6
Position: 18485 -> 10.1111/apt.15951
Position: 18486 -> 10.1111/jan.14718
Position: 18487 -> 10.1007/s42452-020-3173-6
Position: 18488 -> 10.1186/s12917-020-02730-8
Position: 18489 -> 10.1021/acs.jpcb.0c05621
Position: 18490 -> 10.3390/ijerph17176317
Position: 18491 -> 10.1101/2020.05.01.20087023
Position: 18492 -> 10.1101/2021.01.08.423993
Position: 18493 -> 10.1007/978-3-030-53956-6_31
Position: 18494 -> 10.2144/btn-2020-0096
Position: 18495 -> 10.1056/nejmc2008646
Position: 18496 -> 10.1101/2021.01.24.21250418
Position: 18497 -> 10.1016/j.jinf.2020.09.010
Position: 18498 -> 10.7326/m

Position: 18658 -> 10.1017/dmp.2020.313
Position: 18659 -> 10.1101/2020.05.08.20095968
Position: 18660 -> 10.1093/infdis/jiy018
Position: 18661 -> 10.1016/j.jtos.2020.08.003
Position: 18662 -> 10.1002/jmv.26682
Position: 18663 -> 10.3390/v11010058
Position: 18664 -> 10.1016/j.socscimed.2020.113441
Position: 18665 -> 10.1016/j.rmed.2020.106120
Position: 18666 -> 10.3390/jcm9113521
Position: 18667 -> 10.1016/j.jen.2020.06.009
Position: 18668 -> 10.1023/a:1016378226861
Position: 18669 -> 10.1101/2020.07.04.20145870
Position: 18670 -> 10.3390/ijms21103643
Position: 18671 -> 10.1080/07391102.2020.1850360
Position: 18672 -> 10.1016/j.virusres.2017.08.001
Position: 18673 -> 10.1128/jvi.01734-19
Position: 18674 -> 10.1016/j.diagmicrobio.2020.115181
Position: 18675 -> 10.1101/2020.04.11.037473
Position: 18676 -> 10.1101/2020.09.24.20200766
Position: 18677 -> 10.1159/000509065
Position: 18678 -> 10.3390/ijerph18020580
Position: 18679 -> 10.1007/978-3-030-52200-1_1
Position: 18680 -> 10.3390/ijer

Position: 18841 -> 10.1101/2020.05.10.20097295
Position: 18842 -> 10.1016/j.bios.2013.07.037
Position: 18843 -> 10.1007/s00436-020-06786-1
Position: 18844 -> 10.3390/biology9090296
Position: 18845 -> 10.1111/sifp.12138
Position: 18846 -> 10.1093/jpids/piaa162
Position: 18847 -> 10.1136/bmjopen-2020-039088
Position: 18848 -> 10.1016/j.dcn.2020.100899
Position: 18849 -> 10.1684/ecn.2020.0448
Position: 18850 -> 10.1101/2020.05.28.122663
Position: 18851 -> 10.1002/adma.202005637
Position: 18852 -> 10.3399/bjgpopen20x101069
Position: 18853 -> 10.1016/j.meegid.2020.104556
Position: 18854 -> 10.3390/cancers13020266
Position: 18855 -> 10.1093/ibd/izaa303
Position: 18856 -> 10.1111/ajt.16346
Position: 18857 -> 10.3390/jof2020016
Position: 18858 -> 10.1101/2020.10.10.20210427
Position: 18859 -> 10.1007/s42438-020-00128-1
Position: 18860 -> 10.1007/978-3-030-50433-5_33
Position: 18861 -> 10.2147/jpr.s285835
Position: 18862 -> 10.1016/j.xcrm.2020.100059
Position: 18863 -> 10.1038/s41598-018-20305-

Position: 19024 -> 10.1016/j.ijheh.2020.113671
Position: 19025 -> 10.3389/fimmu.2019.01821
Position: 19026 -> 10.1007/s10393-016-1183-z
Position: 19027 -> 10.1186/s12889-020-09079-5
Position: 19028 -> 10.1038/s41598-020-65405-2
Position: 19029 -> 10.1101/2020.03.24.20042408
Position: 19030 -> 10.1101/2020.03.23.20041608
Position: 19031 -> 10.1016/j.heliyon.2020.e04929
Position: 19032 -> 10.1101/2020.11.17.20233452
Position: 19033 -> 10.1016/j.jmb.2017.04.011
Position: 19034 -> 10.1016/j.jclinepi.2020.12.002
Position: 19035 -> 10.1016/j.ijid.2020.05.071
Position: 19036 -> 10.1007/s10753-014-0048-2
Position: 19037 -> 10.1016/j.cell.2019.04.035
Position: 19038 -> 10.1007/978-3-030-49165-9_7
Position: 19039 -> 10.1016/b978-0-12-801477-6.00010-1
Position: 19040 -> 10.1111/j.1750-2659.2009.00095.x
Position: 19041 -> 10.1371/journal.pbio.0030324
Position: 19042 -> 10.1186/s12917-018-1756-x
Position: 19043 -> 10.1365/s40702-020-00684-x
Position: 19044 -> 10.1177/0300060520977593
Position: 1904

Position: 19203 -> 10.1007/s12098-020-03311-1
Position: 19204 -> 10.1007/s10096-020-04137-7
Position: 19205 -> 10.1007/s10479-020-03695-5
Position: 19206 -> 10.1007/s10072-020-04693-y
Position: 19207 -> 10.1007/s00701-020-04544-x
Position: 19208 -> 10.1016/j.tet.2020.131373
Position: 19209 -> 10.1093/ofid/ofab001
Position: 19210 -> 10.1111/j.1600-0625.2006.00527.x
Position: 19211 -> 10.1038/srep24257
Position: 19212 -> 10.1101/2020.05.13.092759
Position: 19213 -> 10.1002/iroh.200510822
Position: 19214 -> 10.1016/j.endien.2020.09.004
Position: 19215 -> 10.1007/s38311-020-0612-3
Position: 19216 -> 10.1007/s00580-010-1084-4
Position: 19217 -> 10.1007/s10875-020-00764-z
Position: 19218 -> 10.1016/j.vaccine.2008.01.006
Position: 19219 -> 10.1101/2020.05.27.117127
Position: 19220 -> 10.1016/j.jep.2020.113319
Position: 19221 -> 10.1016/j.jth.2020.100942
Position: 19222 -> 10.1111/tbed.13419
Position: 19223 -> 10.1016/j.trip.2020.100222
Position: 19224 -> 10.3389/fnhum.2020.598435
Position: 19

...

Position: 22651 -> 10.3390/cancers13010091
Position: 22652 -> 10.1101/2020.06.08.20125872
Position: 22653 -> 10.1016/j.fitote.2014.09.018
Position: 22654 -> 10.1371/journal.pone.0243885
Position: 22655 -> 10.1101/2020.05.19.20107144
Position: 22656 -> 10.1016/j.yamp.2020.07.002
Position: 22657 -> 10.3390/children7090125
Position: 22658 -> 10.1101/2020.12.01.20241695
Position: 22659 -> 10.1186/s13054-020-03241-6
Position: 22660 -> 10.1007/bf03184258
Position: 22661 -> 10.1186/s12955-020-01352-w
Position: 22662 -> 10.1007/s10822-020-00327-9
Position: 22663 -> 10.1016/j.jviromet.2021.114070
Position: 22664 -> 10.1002/jmrs.406
Position: 22665 -> 10.1371/journal.pntd.0008972
Position: 22666 -> 10.3390/jcm9113624
Position: 22667 -> 10.1017/s0950268820001430
Position: 22668 -> 10.1101/2020.11.17.387902
Position: 22669 -> 10.1016/j.psychres.2020.113252
Position: 22670 -> 10.1111/trf.15869
Position: 22671 -> 10.3390/ijerph18020607
Position: 22672 -> 10.1093/sleep/zsaa258
Position: 22673 -> 10.3

Position: 22833 -> 10.1016/j.jcv.2014.03.021
Position: 22834 -> 10.3389/fpsyg.2020.608677
Position: 22835 -> 10.3390/ma13173744
Position: 22836 -> 10.1016/j.celrep.2018.11.082
Position: 22837 -> 10.1016/j.buildenv.2015.02.041
Position: 22838 -> 10.1038/s41415-020-1543-5
Position: 22839 -> 10.3390/ijerph17228701
Position: 22840 -> 10.1016/j.canep.2020.101805
Position: 22841 -> 10.1007/s11423-020-09913-y
Position: 22842 -> 10.1016/j.autrev.2020.102729
Position: 22843 -> 10.3390/s20195603
Position: 22844 -> 10.1016/j.chom.2020.08.004
Position: 22845 -> 10.1101/2020.04.28.20083329
Position: 22846 -> 10.1371/journal.pone.0240150
Position: 22847 -> 10.1101/2020.11.11.20229765
Position: 22848 -> 10.1101/2020.10.30.20222786
Position: 22849 -> 10.1007/978-3-030-47436-2_10
Position: 22850 -> 10.1038/nprot.2006.452
Position: 22851 -> 10.1101/2020.04.06.028647
Position: 22852 -> 10.1016/j.virol.2014.09.020
Position: 22853 -> 10.12688/wellcomeopenres.16225.2
Position: 22854 -> 10.1021/acs.jpclett.0

Position: 23014 -> 10.1101/2020.11.08.373621
Position: 23015 -> 10.3389/fimmu.2020.02037
Position: 23016 -> 10.1007/s10140-020-01865-3
Position: 23017 -> 10.1128/jcm.00875-20
Position: 23018 -> 10.1007/978-3-540-72665-4_8
Position: 23019 -> 10.1101/2020.06.17.20134254
Position: 23020 -> 10.1016/j.worlddev.2018.08.013
Position: 23021 -> 10.3389/fmicb.2019.00419
Position: 23022 -> 10.1016/j.psychres.2020.113525
Position: 23023 -> 10.26434/chemrxiv.12275705
Position: 23024 -> 10.1038/s41467-020-19891-7
Position: 23025 -> 10.1016/j.idh.2020.09.005
Position: 23026 -> 10.1016/j.dci.2011.11.001
Position: 23027 -> 10.1016/j.dsx.2020.05.023
Position: 23028 -> 10.1016/j.dib.2020.106446
Position: 23029 -> 10.1016/j.admp.2008.03.025
Position: 23030 -> 10.1101/2020.06.24.20139006
Position: 23031 -> 10.1007/bf01379081
Position: 23032 -> 10.15585/mmwr.mm6948a3
Position: 23033 -> 10.1371/journal.pgen.1004403
Position: 23034 -> 10.1101/2020.04.25.20079111
Position: 23035 -> 10.3390/bs10120183
Position:

Position: 23196 -> 10.1016/j.virol.2009.06.006
Position: 23197 -> 10.1136/archdischild-2020-320030
Position: 23198 -> 10.1016/j.vetpar.2019.04.008
Position: 23199 -> 10.1186/s41256-020-00164-1
Position: 23200 -> 10.3390/ijerph17249326
Position: 23201 -> 10.1177/0194599820934380
Position: 23202 -> 10.1007/s10459-020-10010-1
Position: 23203 -> 10.1007/978-3-030-45439-5_10
Position: 23204 -> 10.12688/hrbopenres.13031.1
Position: 23205 -> 10.1007/978-3-030-50420-5_36
Position: 23206 -> 10.3389/fpsyg.2020.02011
Position: 23207 -> 10.1016/j.paid.2020.110398
Position: 23208 -> 10.1186/s13071-020-04274-0
Position: 23209 -> 10.1016/j.gendis.2021.01.008
Position: 23210 -> 10.1016/j.ajic.2020.06.190
Position: 23211 -> 10.1128/jvi.00902-20
Position: 23212 -> 10.1101/2020.06.02.20120295
Position: 23213 -> 10.1186/s12967-020-02405-w
Position: 23214 -> 10.1016/j.gene.2008.01.004
Position: 23215 -> 10.1101/2020.12.01.407411
Position: 23216 -> 10.1016/j.jocn.2020.05.017
Position: 23217 -> 10.4258/hir.2

Position: 23377 -> 10.15585/mmwr.mm6946a2
Position: 23378 -> 10.1016/j.cbi.2020.109348
Position: 23379 -> 10.1016/j.xphs.2020.11.023
Position: 23380 -> 10.1101/2020.04.29.20085472
Position: 23381 -> 10.1007/978-3-8274-2156-2_4
Position: 23382 -> 10.1101/2020.10.13.20211821
Position: 23383 -> 10.1101/2020.02.16.951723
Position: 23384 -> 10.1101/2020.11.24.20235887
Position: 23385 -> 10.1023/a:1008037916674
Position: 23386 -> 10.1007/s00430-019-00610-z
Position: 23387 -> 10.1101/2020.04.19.20070870
Position: 23388 -> 10.1002/ddr.10378
Position: 23389 -> 10.1101/2020.08.19.20178137
Position: 23390 -> 10.1007/s00264-009-0829-7
Position: 23391 -> 10.1136/bmjopen-2020-040649
Position: 23392 -> 10.3390/s21020402
Position: 23393 -> 10.2196/10905
Position: 23394 -> 10.1016/j.ausmj.2020.07.001
Position: 23395 -> 10.3390/ijms21051676
Position: 23396 -> 10.1016/j.bmc.2019.115273
Position: 23397 -> 10.1016/j.intimp.2020.106980
Position: 23398 -> 10.1007/s10755-020-09525-7
Position: 23399 -> 10.1016

In [ ]:
dict_new_extra_info

In [ ]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [ ]:
df_new_extra_info

In [ ]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

In [ ]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [ ]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')

In [ ]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

In [ ]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

In [ ]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc